## Define the used fields from the UK biobank, and extract the patients diagnosed with vascular dementia

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DataFields import DataFields
from ProjectFunctions import get_columns_from_chunk

UKBB_PATH = "~/biobank/ukb672220.csv"

VD_COL = "Vascular Dementia Report Date"

# A dictionary containing the used features
datafields = DataFields

In [2]:
# Filter by rows to extract patients with vascular dementia
def filter_by_column(chunk, column):
    # Filter rows where either col_1_filter or col_2_filter is not NaN
    filter_mask = chunk[column].notna()
    filtered_chunk = chunk[filter_mask].copy()
    # Combine dates and add the new columns
    filtered_chunk.loc[:, column] = pd.to_datetime(
        filtered_chunk[column], errors="coerce"
        )

    filtered_chunk.loc[:, column] = filtered_chunk[column].where(
            filtered_chunk[column].between(pd.Timestamp("1950-01-01"), pd.Timestamp("2030-12-31"))
        )
    return filtered_chunk

### Extract rows diagnosed with vascular dementia

In [3]:
chunk_size = 10000
result_df = pd.DataFrame()

for chunk in pd.read_csv(UKBB_PATH, chunksize=chunk_size, low_memory=False):
    
    filtered_chunk = get_columns_from_chunk(chunk, datafields, oldest=True)

    filtered_chunk = filter_by_column(filtered_chunk, VD_COL)
    result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)

# Save the result
result_df.to_csv('diagnosed.csv', index=False)

result_df.describe()

,id,Birth Year,Sex,Education,BMI Impedance,Smoking Status,Ever Smoked,Alcohol Intake Frequency,Report of stroke,Diabetes Diagnosed By Doctor,...,Platelet crit,Platelet distribution width,Red blood cell (erythrocyte) count,Red blood cell (erythrocyte) distribution width,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure
count,2.181000e+03,2181.000000,2181.000000,2132.000000,2157.000000,2176.000000,2154.000000,2176.000000,678.000000,2176.000000,...,2033.000000,2033.000000,2033.000000,2033.000000,2002.000000,2002.000000,2033.000000,1978.000000,1978.000000,1978.000000
mean,3.517504e+06,1943.104998,0.582760,-1.002814,28.563543,0.657629,0.670381,3.213235,9.035398,0.204963,...,0.228823,16.542735,4.489264,13.726473,0.064325,1.435948,7.337614,81.997472,147.262386,70.920627
std,1.441920e+06,4.319199,0.493216,4.964065,5.143175,0.770112,0.470184,1.732589,6.193954,0.429159,...,0.052181,0.532093,0.444526,1.094835,0.035336,0.804845,1.957398,11.211197,21.113931,13.164777
min,1.002012e+06,1936.000000,0.000000,-7.000000,15.757600,-3.000000,0.000000,-3.000000,0.000000,-3.000000,...,0.064000,15.200000,1.720000,11.490000,0.012000,0.263000,2.370000,47.000000,78.000000,35.000000
25%,2.301136e+06,1940.000000,0.000000,-7.000000,25.069400,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.195000,16.200000,4.200000,13.100000,0.045000,1.020000,6.020000,74.000000,133.000000,62.000000
50%,3.520350e+06,1942.000000,1.000000,1.000000,27.805800,1.000000,1.000000,3.000000,11.000000,0.000000,...,0.224000,16.500000,4.484000,13.530000,0.060000,1.330000,7.120000,82.000000,146.000000,70.000000
75%,4.741406e+06,1945.000000,1.000000,3.000000,31.435800,1.000000,1.000000,5.000000,11.000000,0.000000,...,0.257000,16.860000,4.773000,14.100000,0.078000,1.739750,8.410000,89.000000,160.000000,79.000000
max,6.022126e+06,1968.000000,1.000000,6.000000,54.528300,2.000000,1.000000,6.000000,22.000000,1.000000,...,0.624000,19.400000,6.390000,31.700000,1.077000,25.278000,34.130000,132.000000,241.000000,169.000000
